# Preprocessing and Pivoting Netflix Prize Data

We will be taking our combined data set and reducing the size by reducing to smaller data types, and then pivoting the dataframe before using it to create a sparse dataframe with pandas. This reduces the size of the dataframe to approximately 1.1 GB from 2.6 GB. 
Next, we will use Stochastic Gradient Descent to find the rank-40 SVD of the full matrix, ignoring the empty values, which allows us to predict ratings with for the empty values.

We will also use a Nearest Neighborhood algorithm for our collaborative filtering. Thsi requires creating a similarity function that subtracts a users individual average of all items (not including null values), then add it back for the target user. From there, we can use the neighbors to predict a rating.

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
cwd = os.getcwd()
movie = pd.read_csv(cwd + "/data/final.csv")

movie.describe()

In [3]:
movie.head()

,MovieID,CustomerID,Rating,Date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


## Reducing Data Size

We will be changing the Date object to 'category', as well as reducing the data type of the rest of the columns from int64 to int32 for the IDs, and to int8 for the ratings.

In [4]:
movie.memory_usage()

Index               128
MovieID       803844056
CustomerID    803844056
Rating        803844056
Date          803844056
dtype: int64

In [5]:
movie.memory_usage().sum() / (1024**2)

3066.421844482422

In [6]:
movie.dtypes

MovieID        int64
CustomerID     int64
Rating         int64
Date          object
dtype: object

In [3]:
movie['Date'] = movie['Date'].astype('category')
movie['MovieID'] = movie['MovieID'].astype('int16')
movie['CustomerID'] = movie['CustomerID'].astype('int32')
movie['Rating'] = movie['Rating'].astype('int8')

In [9]:
movie.memory_usage().sum() / (1024**2)

862.5109052658081

## Changing into a Sparse Dataframe

First, we must pivot the dataframe so that it is in the form of a sparse dataframe. However, in order to pivot, we need a far smaller dataframe. So first we sort, divide, pivot, and then merge back together.

In [4]:
import pickle
from tqdm import tqdm

In [5]:
movie.sort_values(by = "MovieID", inplace = True)

In [12]:
movie.memory_usage().sum() / (1024**2)

1629.116213798523

In [9]:
chunk_size = 12560063
chunks = [x for x in range(0, movie.shape[0], chunk_size)]

smovie = pd.DataFrame()
nmovie = np.array([])
for i in tqdm(range(0, len(chunks) - 1)):
    chunk_movie = movie.iloc[chunks[i]:chunks[i + 1] - 1]
    pmovie = chunk_movie.pivot_table(values='Rating', index='CustomerID', columns='MovieID')
    #pmovie.to_numpy().save("numpy" + str(i), allow_pickle=True)
    np.save("numpy" + str(i), pmovie.to_numpy(), allow_pickle=True)
    #nmovie = np.append(nmovie, pmovie.to_numpy())
    #smovie = smovie.append(pmovie.astype(pd.SparseDtype("int8", np.nan)))

100%|██████████| 8/8 [09:42<00:00, 72.80s/it]


In [16]:
smovie.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
CustomerID,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
smovie.to_pickle("sparse")

## Test-Train Split

We need to split our data into an 80-20 Training-Testing split. This lowers the data usage, as well as lets us analyze the data.

In [6]:
from sklearn.model_selection import train_test_split

In [14]:
# for each loop to append numpy array
# get movie and customer id from sparse file
s_movie = np.load("numpy0.npy")

In [12]:
split = int(s_movie.shape[0]*0.8)
movie_train = s_movie.iloc[:split]
movie_test = s_movie.iloc[split + 1:]

In [13]:
movie_train.to_pickle("sparseTrain")
movie_test.to_pickle("sparseTest")

Save our new training and testing sparse data sets to separate pickle files for easier ease of future use.

### Find Global Average, User Average, and Movie Average

In [25]:
nonzeros = movie_train.eq(0).sum(axis=1).count
print(nonzeros)

<bound method Series.count of CustomerID
6          0
7          0
8          0
10         0
25         0
          ..
1088922    0
1088924    0
1088925    0
1088932    0
1088949    0
Length: 2961549, dtype: int64>


In [26]:
print(movie_train.sum())

MovieID
1        2051.0
2         516.0
3        7326.0
4         389.0
5        4468.0
          ...  
17766       0.0
17767       0.0
17768       0.0
17769       0.0
17770       0.0
Length: 17770, dtype: float64


In [31]:
globalAvg = s_movie.sum()/s_movie.count_nonzero()
print("Global Average Rating {}".format(globalAvg))

AttributeError: 'DataFrame' object has no attribute 'count_nonzero'

Create a loss function and optimize for P and Q? Follow Blog post for instructions on getting ratings from it. Also attempt to simply use NMF from python.